In [34]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, KFold
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_score, recall_score, f1_score


# Загрузка и подготовка данных

In [35]:
file_path = 'bank-full.csv'
df = pd.read_csv(file_path, sep=';')

# Выбор необходимых столбцов

In [36]:
columns = ['age', 'job', 'marital', 'education', 'balance', 'housing', 'contact',
           'day', 'month', 'duration', 'campaign', 'pdays', 'previous', 'poutcome', 'y']
df = df[columns]

# Преобразование целевой переменной 'y' в 0/1

In [37]:
df['y'] = (df['y'] == 'yes').astype(int)


# Разделение данных (60/20/20) с random_state=1


In [38]:
df_full_train, df_test = train_test_split(df, test_size=0.2, random_state=1)
df_train, df_val = train_test_split(df_full_train, test_size=0.25, random_state=1) # 0.25 от 80% = 20%



# Сброс индексов и разделение на признаки/целевую переменную


In [39]:
df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)
df_full_train = df_full_train.reset_index(drop=True)

y_train = df_train['y'].values
y_val = df_val['y'].values
y_full_train = df_full_train['y'].values

del df_train['y']
del df_val['y']
del df_full_train['y']

numerical = ['age', 'balance', 'day', 'duration', 'campaign', 'pdays', 'previous']
categorical = ['job', 'marital', 'education', 'housing', 'contact', 'month', 'poutcome']

print(f"Размер обучающей выборки: {len(df_train)}")
print(f"Размер валидационной выборки: {len(df_val)}")
print(f"Размер полного обучающего набора (train+val): {len(df_full_train)}")
print("-" * 30)

Размер обучающей выборки: 27126
Размер валидационной выборки: 9042
Размер полного обучающего набора (train+val): 36168
------------------------------


# ВОПРОС 1: ВАЖНОСТЬ ПРИЗНАКОВ ПО ROC AUC

In [40]:
numerical_q1 = ['balance', 'day', 'duration', 'previous']
aucs = {}

for col in numerical_q1:
    auc = roc_auc_score(y_train, df_train[col])

    if auc < 0.5:
        auc = roc_auc_score(y_train, -df_train[col])

    aucs[col] = auc

best_auc_q1 = max(aucs, key=aucs.get)
print(f"AUCs: {aucs}")
print(f"Ответ 1 (Наивысший AUC): {best_auc_q1}")
print("-" * 30)

AUCs: {'balance': np.float64(0.5888313805382317), 'day': np.float64(0.525957882383908), 'duration': np.float64(0.8147002759670778), 'previous': np.float64(0.5985653242764153)}
Ответ 1 (Наивысший AUC): duration
------------------------------


# --- 3. ВОПРОС 2: ОБУЧЕНИЕ МОДЕЛИ И AUC ---

In [41]:
dv = DictVectorizer(sparse=False)

train_dicts = df_train.to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val.to_dict(orient='records')
X_val = dv.transform(val_dicts)

### Обучение модели


In [42]:
model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=1)
model.fit(X_train, y_train)

LogisticRegression(max_iter=1000, random_state=1, solver='liblinear')

### Предсказание на валидационной выборке


In [43]:
y_pred = model.predict_proba(X_val)[:, 1]

### Вычисление AUC


In [44]:
auc_val = roc_auc_score(y_val, y_pred)
print(f"AUC на валидационном наборе: {auc_val:.3f}")

answer_2 = round(auc_val, 2)
print(f"Ответ 2 (AUC): {answer_2}")
print("-" * 30)

AUC на валидационном наборе: 0.900
Ответ 2 (AUC): 0.9
------------------------------


# ВОПРОС 3

In [45]:
thresholds = np.linspace(0.0, 1.0, 101)
scores = []

for t in thresholds:
    y_predict_threshold = (y_pred >= t)

    P = precision_score(y_val, y_predict_threshold, zero_division=0)
    R = recall_score(y_val, y_predict_threshold, zero_division=0)
    F1 = 2 * (P * R) / (P + R) if (P + R) > 0 else 0

    scores.append((t, P, R, F1))

df_scores = pd.DataFrame(scores, columns=['threshold', 'precision', 'recall', 'f1'])

df_scores['diff'] = np.abs(df_scores['precision'] - df_scores['recall'])
intersection_point = df_scores.sort_values(by='diff').iloc[0]
answer_3 = intersection_point['threshold']

print(f"Порог, где P и R пересекаются: {answer_3:.3f}")
print(f"Ответ 3 (Порог пересечения P/R): {answer_3:.2f}")


Порог, где P и R пересекаются: 1.000
Ответ 3 (Порог пересечения P/R): 1.00


# ВОПРОС 4

In [46]:
max_f1_score = df_scores.sort_values(by='f1', ascending=False).iloc[0]
answer_4 = max_f1_score['threshold']

print(f"Порог максимального F1: {answer_4:.3f}")
print(f"Ответ 4 (Порог макс. F1): {answer_4:.2f}")
print("-" * 30)

Порог максимального F1: 0.220
Ответ 4 (Порог макс. F1): 0.22
------------------------------


# ВОПРОС 5

In [47]:
X_full_train = df_full_train.copy()
kfold = KFold(n_splits=5, shuffle=True, random_state=1)
auc_scores_q5 = []

model_params = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=1)

for train_idx, val_idx in kfold.split(X_full_train):
    df_train_fold = X_full_train.iloc[train_idx]
    y_train_fold = y_full_train[train_idx]

    df_val_fold = X_full_train.iloc[val_idx]
    y_val_fold = y_full_train[val_idx]

    dv_fold = DictVectorizer(sparse=False)
    X_train_fold = dv_fold.fit_transform(df_train_fold.to_dict(orient='records'))
    X_val_fold = dv_fold.transform(df_val_fold.to_dict(orient='records'))

    model_fold = model_params
    model_fold.fit(X_train_fold, y_train_fold)

    y_pred_fold = model_fold.predict_proba(X_val_fold)[:, 1]
    auc_scores_q5.append(roc_auc_score(y_val_fold, y_pred_fold))

std_auc = np.std(auc_scores_q5)
answer_5 = round(std_auc, 3)

print(f"AUC на 5 фолдах: {auc_scores_q5}")
print(f"Стандартное отклонение AUC: {std_auc:.4f}")
print(f"Ответ 5 (Стандартное отклонение): {answer_5}")
print("-" * 30)

AUC на 5 фолдах: [np.float64(0.8992842405563055), np.float64(0.8990110164487815), np.float64(0.9111641244449282), np.float64(0.9078905031338849), np.float64(0.9119728949040351)]
Стандартное отклонение AUC: 0.0057
Ответ 5 (Стандартное отклонение): 0.006
------------------------------


# ВОПРОС 6

In [48]:
print("--- 6. ВОПРОС 6: ТЮНИНГ ГИПЕРПАРАМЕТРОВ (C) ---")

C_values = [0.000001, 0.001, 1]
results = []
kfold_q6 = KFold(n_splits=5, shuffle=True, random_state=1)

for C in C_values:
    auc_scores = []

    model_params = LogisticRegression(solver='liblinear', C=C, max_iter=1000, random_state=1)

    for train_idx, val_idx in kfold_q6.split(X_full_train):
        df_train_fold = X_full_train.iloc[train_idx]
        y_train_fold = y_full_train[train_idx]
        df_val_fold = X_full_train.iloc[val_idx]
        y_val_fold = y_full_train[val_idx]

        dv_fold = DictVectorizer(sparse=False)
        X_train_fold = dv_fold.fit_transform(df_train_fold.to_dict(orient='records'))
        X_val_fold = dv_fold.transform(df_val_fold.to_dict(orient='records'))

        model_fold = model_params
        model_fold.fit(X_train_fold, y_train_fold)
        y_pred_fold = model_fold.predict_proba(X_val_fold)[:, 1]
        auc_scores.append(roc_auc_score(y_val_fold, y_pred_fold))

    mean_auc = np.mean(auc_scores)
    std_auc = np.std(auc_scores)
    results.append((C, mean_auc, std_auc))

--- 6. ВОПРОС 6: ТЮНИНГ ГИПЕРПАРАМЕТРОВ (C) ---


### Поиск лучшего C: макс. mean_auc, затем мин. std_auc, затем мин. C


In [49]:
best_C = None
best_mean_auc = -1
best_std_auc = float('inf')

print("Результаты CV для разных C:")
for C, mean_auc, std_auc in results:
    print(f"C={C:<8}: mean_auc={mean_auc:.4f}, std={std_auc:.4f}")

    if mean_auc > best_mean_auc:
        best_mean_auc = mean_auc
        best_std_auc = std_auc
        best_C = C
    elif mean_auc == best_mean_auc:
        if std_auc < best_std_auc:
            best_std_auc = std_auc
            best_C = C
        elif std_auc == best_std_auc and C < best_C:
            best_C = C

answer_6 = best_C

print(f"Ответ 6 (Лучшее C): {answer_6}")
print("-" * 30)

Результаты CV для разных C:
C=1e-06   : mean_auc=0.7014, std=0.0094
C=0.001   : mean_auc=0.8608, std=0.0072
C=1       : mean_auc=0.9059, std=0.0057
Ответ 6 (Лучшее C): 1
------------------------------



# Финальные результаты

Вопросы       | Ответы
------------- | -------------
Вопрос 1 - (Признак с макс. AUC)  | duration
Вопрос 2 (AUC на валидации)  | 0.9
Вопрос 3 (Порог P/R пересечения)  | 1.00
Вопрос 4 (Порог макс. F1)  | 0.22
Вопрос 5 (Стандартное откл. AUC CV)  | 0.006
Вопрос 6 (Лучший C)  | 1